# Prepare Forecasts
The forecasts of the forecasting model contain more information than necessary. Here, we save them again with only the relevant information. This might seem overkill for a simple deterministic approach, but allows for easier adjustments (eg when investigating probabilistic forecasts).

In [ ]:
from pathlib import Path
import pandas as pd
import os
import re

def store_det_fc(df, csv_path, creation_time=False):
    """ Store the deterministic forecasts to a CSV file. If the file already exists, a counter is appended to the filename.
    
    Args:
        df (pd.DataFrame): DataFrame containing the deterministic forecasts and other information.
        csv_path (str): Original CSV file path to base the new filename and directory on.
        creation_time (bool): If True, add the time of execution to the filename.
    """

    # Create the filename and directory based on the original CSV path
    filename = os.path.basename(csv_path).replace('file_fc', 'file_fc_parametric')
    if creation_time:
        # add current timestamp to the filename
        current_time = pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M')
        filename = filename.replace('.csv', f'_CreationTime{current_time}.csv')
    directory = os.path.dirname(csv_path).replace('storage_quantile_fc', 'storage_param_fc')

    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    # Prepare the filepath
    filepath = os.path.join(directory, filename)
    base, ext = os.path.splitext(filename)
    counter = 1

    # Find the '_SFH' pattern in the filename (e.g., 'file_fc_SFH3_2024-07-25.csv')
    pattern = r'(_SFH\d+)'
    match = re.search(pattern, base)
    sfh_part = match.group(1)

    while os.path.exists(filepath):
        # Insert the counter before _SFH part
        new_base = base.replace(sfh_part, f"{counter:02d}{sfh_part}")
        filepath = os.path.join(directory, f"{new_base}{ext}")
        counter += 1

    # Save to CSV
    df.to_csv(filepath, index=True)
    print(f"Deterministic forecasts saved to {filepath}")

In [ ]:
time_of_fc_creation = '2025-08-16_18-35-40' # enter the date_time of forecasts to be processed
path_list = list(Path('mount/storage_quantile_fc/').glob(f'**/file_fc*_{time_of_fc_creation}_freq*.csv'))

for path in path_list:
    df = pd.read_csv(path, index_col=[0,1], usecols=['time_fc_created','timestamp', 'P_TOT_HAT'])
    df = df.rename(columns={'P_TOT_HAT': 'expected_value'})
    store_det_fc(df, path)

Parametric forecasts saved to mount/storage_param_fc/SFH3/2025-08-16_18-35-40/file_fc_parametric_AutoKANDet_MSE_SFH3_2025-08-16_18-35-40_freq60.csv
